In [ ]:
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
from sagemaker import Session
from sagemaker.estimator import Estimator
from sagemaker.amazon.amazon_estimator import get_image_uri

In [ ]:
# Define AWS resources
s3_bucket = "recommender-system-demo"
s3_prefix = "training-data"
sagemaker_role = "arn:aws:iam::YOUR_ACCOUNT_ID:role/service-role/AmazonSageMaker-ExecutionRole"

# Initialize boto3 clients
s3_client = boto3.client('s3')


In [ ]:
# Step 1: Load and preprocess data
def load_and_preprocess_data():
    # Load dataset
    movies_file = f"s3://{s3_bucket}/processed/movies.csv"
    ratings_file = f"s3://{s3_bucket}/processed/ratings.csv"

    movies_df = pd.read_csv(movies_file)
    ratings_df = pd.read_csv(ratings_file)

    # Merge datasets
    data = pd.merge(ratings_df, movies_df, on="movieId")
    
    # Prepare for training
    data = data[["userId", "movieId", "rating"]]

    # Encode user_id and movie_id as integers
    data["userId"] = data["userId"].astype("category").cat.codes
    data["movieId"] = data["movieId"].astype("category").cat.codes

    # Split data
    train, test = train_test_split(data, test_size=0.2, random_state=42)

    # Save preprocessed data to S3
    train.to_csv("train.csv", index=False)
    test.to_csv("test.csv", index=False)

    s3_client.upload_file("train.csv", s3_bucket, f"{s3_prefix}/train/train.csv")
    s3_client.upload_file("test.csv", s3_bucket, f"{s3_prefix}/test/test.csv")

    print("Data preprocessing completed and uploaded to S3.")

In [ ]:

# Step 2: Train model
def train_model():
    session = Session()

    container = get_image_uri(boto3.Session().region_name, "factorization-machines")
    
    # Define estimator
    estimator = Estimator(
        container,
        role=sagemaker_role,
        instance_count=1,
        instance_type="ml.m5.large",
        output_path=f"s3://{s3_bucket}/output"
    )

    # Specify input data
    train_input = f"s3://{s3_bucket}/{s3_prefix}/train"
    test_input = f"s3://{s3_bucket}/{s3_prefix}/test"

    estimator.fit({
        "train": train_input,
        "test": test_input
    })

    print("Model training completed.")


In [ ]:
# Step 3: Deploy model
def deploy_model():
    # Deploy the model
    predictor = estimator.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name="recommender-endpoint"
    )

    print(f"Model deployed at endpoint: recommender-endpoint")
    return predictor

# Execute the pipeline
if __name__ == "__main__":
    load_and_preprocess_data()
    train_model()
    deploy_model()